In [ ]:
import pandas as pd
import tensorflow as tf


import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import time
import string


from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go

import os



/kaggle/input/english-urdu/Quran-UR (1)
/kaggle/input/english-urdu/Quran-EN (1)


In [ ]:
with open('/content/Quran-EN (1)', 'r', encoding='utf-8') as file:
    english_text = file.read().splitlines()

with open('/content/Quran-UR (1)', 'r', encoding='utf-8') as file:
    urdu_text = file.read().splitlines()

In [ ]:
print("Englist Text records: ",len(english_text), ', Urdu Text records: ' ,len(urdu_text))

Englist Text records:  6414 , Urdu Text records:  6414


In [ ]:
exclude = set(string.punctuation) #
remove_digits = str.maketrans('', '', string.digits)

In [ ]:
def preprocess_eng_sentence(sent):
    '''Function to preprocess English sentence'''
    sent = sent.lower()
    sent = re.sub("'", '', sent)
    sent = ''.join(ch for ch in sent if ch not in exclude)
    sent = sent.translate(remove_digits)
    sent = sent.strip()
    sent = re.sub(" +", " ", sent)
    sent = '<start> ' + sent + ' <end>'
    return sent

In [ ]:
urdu_diacritics  = ['ِ', 'ٰ', 'ُ', 'ٍ', 'ً', 'َ']

def remove_diacritics(text):
    for letter in text:

        if letter in urdu_diacritics:
            text = text.replace(letter, '')
    return text



In [ ]:
urdu_digits = ['۶', '۴', '۵', '۸', '۲', '۰', '۷', '۹', '۳', '۱']

english_digits=['1','2','3','4','5','6','7','8','9','0']

def remove_numbers(text):
    for letter in text:
        if letter in urdu_digits or letter in english_digits :
            text = text.replace(letter, '')
    return text

In [ ]:
def preprocess_ur_sentence(sent):
  sent = re.sub("'", '', sent)
  sent = remove_diacritics(sent)
  sent = re.sub(r'[؛۔٫٪+=@#!؟،۔)(}{]', '',sent)
  sent = remove_numbers(sent)
  sent = sent.strip()
  sent = re.sub(" +", " ", sent)
  sent = '<start> ' + sent + ' <end>'
  return sent

In [ ]:
sent_pairs = []
for eng, ur in zip(english_text, urdu_text):
    eng = preprocess_eng_sentence(eng)
    ur = preprocess_ur_sentence(ur)
    sent_pair = [eng, ur]
    sent_pairs.append(sent_pair)
sent_pairs[5000:5010]

[['<start> and before that he destroyed the people of nuh noah as well surely they were extremely wicked and exceedingly defiant <end>',
  '<start> اور اس سے پہلے قوم نوح کو بھی ہلاک کیا بیشک وہ بڑے ہی ظالم اور بڑے ہی سرکش تھے <end>'],
 ['<start> and he is the one who raised up the overturned towns of the people of lut lot and smashed them down <end>',
  '<start> اور قوم لوط کی الٹی ہوئی بستیوں کو اوپر اٹھا کر اسی نے نیچے دے پٹکا <end>'],
 ['<start> then that covered them which did cover i e the stones were rained on them <end>',
  '<start> پس ان کو ڈھانپ لیا جس نے ڈھانپ لیا یعنی پھر ان پر پتھروں کی بارش کر دی گئی <end>'],
 ['<start> so o man which of the favours of your lord will you doubt <end>',
  '<start> سو اے انسان تو اپنے پروردگار کی کن کن نعمتوں میں شک کرے گا <end>'],
 ['<start> this holy prophet blessings and peace be upon him is also a warner of the warners of old <end>',
  '<start> یہ رسول اکرم صلی اللہ علیہ وآلہ وسلم بھی اگلے ڈر سنانے والوں میں سے ایک ڈر سنانے والے ہیں <end

In [ ]:

class LanguageIndex():
    def __init__(self, lang):
        self.lang = lang
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()

        self.create_index()

    def create_index(self):
        for phrase in self.lang:
            self.vocab.update(phrase.split(' '))

        self.vocab = sorted(self.vocab)

        self.word2idx['<pad>'] = 0
        for index, word in enumerate(self.vocab):
            self.word2idx[word] = index + 1

        for word, index in self.word2idx.items():
            self.idx2word[index] = word

In [ ]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [ ]:
def load_dataset(pairs, num_examples):

    inp_lang = LanguageIndex(en for en, ma in pairs)
    targ_lang = LanguageIndex(ma for en, ma in pairs)

    input_tensor = [[inp_lang.word2idx[s] for s in en.split(' ')] for en, ma in pairs]


    target_tensor = [[targ_lang.word2idx[s] for s in ma.split(' ')] for en, ma in pairs]


    max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)


    input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor,
                                                                 maxlen=max_length_inp,
                                                                 padding='post')

    target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor,
                                                                  maxlen=max_length_tar,
                                                                  padding='post')

    return input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_tar

In [ ]:
input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_targ = load_dataset(sent_pairs, len(english_text))

### Creating training and validation sets using an 80-20 split

In [ ]:

input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.1, random_state = 101)

len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

(5772, 5772, 642, 642)

In [ ]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 4
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 32
units = 64
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
def gru(units):

    return tf.keras.layers.GRU(units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_activation='sigmoid',
                                   recurrent_initializer='glorot_uniform')


In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units, return_sequences=True, return_state=True)

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state


    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units, return_sequences=True, return_state=True)
        self.fc = tf.keras.layers.Dense(vocab_size)


        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, x, hidden, enc_output):
        hidden_with_time_axis = tf.expand_dims(hidden, 1)


        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))


        attention_weights = tf.nn.softmax(score, axis=1)


        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)


        x = self.embedding(x)


        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)


        output, state = self.gru(x)


        output = tf.reshape(output, (-1, output.shape[2]))


        x = self.fc(output)

        return x, state, attention_weights




    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [ ]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

In [ ]:
optimizer = tf.optimizers.Adam()

def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    return tf.reduce_mean(loss_)

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [ ]:
EPOCHS = 5

for epoch in range(EPOCHS):
    start = time.time()

    hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset):
        loss = 0

        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp, hidden)

            dec_hidden = enc_hidden

            dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * BATCH_SIZE, 1)


            for t in range(1, targ.shape[1]):

                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

                loss += loss_function(targ[:, t], predictions)


                dec_input = tf.expand_dims(targ[:, t], 1)

        batch_loss = (loss / int(targ.shape[1]))

        total_loss += batch_loss

        variables = encoder.variables + decoder.variables

        gradients = tape.gradient(loss, variables)

        optimizer.apply_gradients(zip(gradients, variables))

        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every epoch
    checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))


Epoch 1 Batch 0 Loss 2.4580
Epoch 1 Batch 100 Loss 1.1422
Epoch 1 Batch 200 Loss 0.3869
Epoch 1 Batch 300 Loss 0.6631
Epoch 1 Batch 400 Loss 0.6247
Epoch 1 Batch 500 Loss 1.0076
Epoch 1 Batch 600 Loss 0.7182
Epoch 1 Batch 700 Loss 0.8851
Epoch 1 Batch 800 Loss 1.2327
Epoch 1 Batch 900 Loss 1.8690
Epoch 1 Batch 1000 Loss 1.0796
Epoch 1 Batch 1100 Loss 0.8217
Epoch 1 Batch 1200 Loss 1.0273
Epoch 1 Batch 1300 Loss 0.8986
Epoch 1 Batch 1400 Loss 1.0471
Epoch 1 Loss 0.9886
Time taken for 1 epoch 3660.0839717388153 sec

Epoch 2 Batch 0 Loss 1.4979
Epoch 2 Batch 100 Loss 1.3642
Epoch 2 Batch 200 Loss 1.0835
Epoch 2 Batch 300 Loss 0.5646
Epoch 2 Batch 400 Loss 0.6851
Epoch 2 Batch 500 Loss 1.1060
Epoch 2 Batch 600 Loss 0.6160
Epoch 2 Batch 700 Loss 0.3555
Epoch 2 Batch 800 Loss 0.6045
Epoch 2 Batch 900 Loss 0.5625
Epoch 2 Batch 1000 Loss 1.1409
Epoch 2 Batch 1100 Loss 0.9630
Epoch 2 Batch 1200 Loss 0.9945
Epoch 2 Batch 1300 Loss 0.8352
Epoch 2 Batch 1400 Loss 0.7268
Epoch 2 Loss 0.8761
Time ta

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
def evaluate(inputs, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):

    attention_plot = np.zeros((max_length_targ, max_length_inp))
    sentence = ''
    for i in inputs[0]:
        if i == 0:
            break
        sentence = sentence + inp_lang.idx2word[i] + ' '
    sentence = sentence[:-1]

    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word2idx['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)

        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.idx2word[predicted_id] + ' '

        if targ_lang.idx2word[predicted_id] == '<end>':
            return result, sentence, attention_plot

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot


In [ ]:
def predict_random_val_sentence():
    actual_sent = ''
    k = np.random.randint(len(input_tensor_val))
    random_input = input_tensor_val[k]
    random_output = target_tensor_val[k]
    random_input = np.expand_dims(random_input,0)
    result, sentence, attention_plot = evaluate(random_input, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)
    print('Input: {}'.format(sentence[8:-6]))
    print('Predicted translation: {}'.format(result[:-6]))
    for i in random_output:
        if i == 0:
            break
        actual_sent = actual_sent + targ_lang.idx2word[i] + ' '
    actual_sent = actual_sent[8:-7]
    print('Actual translation: {}'.format(actual_sent))
    attention_plot = attention_plot[:len(result.split(' '))-2, 1:len(sentence.split(' '))-1]
    sentence, result = sentence.split(' '), result.split(' ')
    sentence = sentence[1:-1]
    result = result[:-2]




In [ ]:
predict_random_val_sentence()

Input: no one except allah can bring it forth and establish
Predicted translation: اور وہ لوگ اس کے لئے ہے 
Actual translation: اﷲ کے سوا اسے کوئی ظاہر اور قائم کرنے والا نہیں ہے
